In [ ]:
# default_exp learner
# default_cls_lvl 3

In [ ]:
#hide
%load_ext line_profiler

In [ ]:
#export
from seqdata.core import *
from seqdata.model import *
from fastai2.basics import *
from fastai2.callback.progress import *
from fastai2.callback.tracker import *

# Learner
> Pytorch Modules for Training Models for sequential data

In [ ]:
seq = DataBlock(blocks=(SequenceBlock.from_hdf(['current','voltage'],TensorSequencesInput,clm_shift=[-1,-1]),
                        SequenceBlock.from_hdf(['voltage'],TensorSequencesOutput,clm_shift=[1])),
                 get_items=CreateDict([DfHDFCreateWindows(win_sz=1000+1,stp_sz=1000,clm='current')]),
                 splitter=ApplyToDict(ParentSplitter()))
db = seq.databunch(get_hdf_files('test_data/'))

In [ ]:
model = SimpleGRU(2,1)
lrn = Learner(db,model,loss_func=nn.MSELoss()).fit(1)

epoch,train_loss,valid_loss,time
0,7.203979,0.807301,00:03


## Callbacks

In [ ]:
#export
class SkipFirstNCallback(Callback):
    "`Callback` skips first n samples from prediction and target, optionally `with_loss`"
    def __init__(self, n_skip = 0):
        self.n_skip = n_skip

    def after_pred(self):
        self.learn.pred = self.pred[:,self.n_skip:]
#         import pdb; pdb.set_trace()
        if isinstance(self.yb, tuple):
            self.learn.yb = tuple([y[:,self.n_skip:] for y in self.yb])
        else:
            self.learn.yb = self.yb[:,self.n_skip:]


In [ ]:
#export
class VarySeqLen(Callback):
    "`Callback` varies sequence length of every mini batch"
    def __init__(self, min_len = 50):
        self.min_len = min_len

    def begin_batch(self):
#         import pdb; pdb.set_trace()
        lx = self.xb[0].shape[1]
        ly = self.yb[0].shape[1]
        lim = random.randint(self.min_len,ly)
        
        if ly < lx:
            self.learn.xb = tuple([x[:,:-(ly-lim)] for x in self.xb])
        else:
            self.learn.xb = tuple([x[:,:lim] for x in self.xb])
            
        self.learn.yb = tuple([y[:,:lim] for y in self.yb])

In [ ]:
#export
from fastai2.callback.hook import *
@delegates()
class TimeSeriesRegularizer(HookCallback):
    "Callback that adds AR and TAR to the loss, calculated by output of provided layer"
    run_before=TrainEvalCallback
    def __init__(self,alpha=0.0, beta=0.0,dim = None,detach=False, **kwargs):
        super().__init__(detach=detach,**kwargs)
        store_attr(self,'alpha,beta,dim')
        
    def hook(self, m, i, o): 
#         import pdb; pdb.set_trace()
        if type(o) is torch.Tensor:
            self.out = o
        else:
            self.out = o[0]
        
        #find time axis if not already provided
        if self.dim is None:
            self.dim = np.argmax([0,self.out.shape[1],self.out.shape[2]])
    
    def after_loss(self):
        if not self.training: return
        
        h = self.out.float()
        
        if self.alpha != 0.:  
            l_a = self.alpha * h.pow(2).mean()
            self.learn.loss += l_a 
            
        if self.beta != 0. and h.shape[self.dim]>1:
            h_diff = (h[:,1:] - h[:,:-1]) if self.dim == 1 else (h[:,:,1:] - h[:,:,:-1])
            l_b = self.beta * h_diff.pow(2).mean()
            self.learn.loss += l_b

In [ ]:
#export
class ARInitCB(Callback):
    '''Adds the target variable to the input tuple for autoregression'''
    def begin_batch(self):
#         import pdb; pdb.set_trace()
        self.learn.xb = tuple([*self.xb,*self.yb])

In [ ]:
Learner(db,model,loss_func=nn.MSELoss()).fit(1)

epoch,train_loss,valid_loss,time
0,0.157280,0.064923,00:03


## Loss Functions

In [ ]:
#export
def SkipNLoss(fn,n_skip=0):
    '''Loss-Function modifier that skips the first n samples of sequential data'''
    def _inner( input, target):
        return fn(input[:,n_skip:],target[:,n_skip:])
    
    #checking if fn has the attribute name leads sometimes to false -> try_catch instead
    try:
        _inner.__name__ = fn.__name__
    except:
        pass
    
    return _inner

In [ ]:
Learner(db,model,loss_func=SkipNLoss(nn.MSELoss(),n_skip=30)).fit(1)

epoch,train_loss,valid_loss,time
0,0.004384,0.000829,00:03


In [ ]:
#export
def fun_rmse(inp, targ): 
    '''rmse loss function defined as a function not as a AccumMetric'''
    return torch.sqrt(F.mse_loss(inp, targ))

In [ ]:
Learner(db,model,loss_func=nn.MSELoss(),metrics=SkipNLoss(fun_rmse,n_skip=30)).fit(1)

epoch,train_loss,valid_loss,fun_rmse,time
0,0.035728,0.027303,0.016717,00:03


In [ ]:
#export
def norm_rmse(inp, targ):
    '''rmse loss function defined as a function not as a AccumMetric'''
    return fun_rmse(inp, targ)*100

In [ ]:
Learner(db,model,loss_func=nn.MSELoss(),metrics=SkipNLoss(norm_rmse,n_skip=30)).fit(1)

epoch,train_loss,valid_loss,norm_rmse,time
0,0.023302,0.019464,1.176338,00:03


In [ ]:
#export
def mean_vaf(inp,targ):
    return (1-((targ-inp).var()/targ.var()))*100

In [ ]:
Learner(db,model,loss_func=nn.MSELoss(),metrics=SkipNLoss(mean_vaf,n_skip=30)).fit(1)

epoch,train_loss,valid_loss,mean_vaf,time
0,0.018317,0.015500,99.835579,00:03


# Create Learner Models
Create Learner with different kinds of models with fitting Parameters and regularizations.

In [ ]:
#export
def get_inp_out_size(db):
    '''returns input and output size of a timeseries databunch'''
    tup = db.one_batch()
    inp = tup[0].shape[-1]
    out = tup[1].shape[-1]
    return inp,out

In [ ]:
test_eq(get_inp_out_size(db),(2,1)) 

## GRU Learner
The Learners include model specific optimizations. Removing the first n_skip samples of the loss function of transient time, greatly improves training stability. In

In [ ]:
#export
@delegates(SimpleGRU, keep=True)
def GRULearner(db,alpha=0,beta=0,early_stop=0,metrics=None,n_skip=0,fname='model',**kwargs):
    inp,out = get_inp_out_size(db)
    model = SimpleGRU(inp,out,**kwargs)
    
    cbs=[TimeSeriesRegularizer(alpha=alpha,beta=beta,modules=[model.rnn]),SaveModelCallback(fname=fname)]
    if early_stop > 0:
        cbs += [EarlyStoppingCallback(patience=early_stop)]
        
    skip = partial(SkipNLoss,n_skip=n_skip)
        
    if metrics is None: metrics=skip(fun_rmse)
        
    lrn = Learner(db,model,loss_func=skip(nn.MSELoss()),opt_func=ranger,metrics=metrics,cbs=cbs)
    return lrn

In [ ]:
GRULearner(db).fit(1)

epoch,train_loss,valid_loss,fun_rmse,time
0,12.178429,11.746172,3.412008,00:04


## QRNN Learner

In [ ]:
#export
@delegates(SimpleQRNN, keep=True)
def QRNNLearner(db,alpha=1,beta=1,early_stop=0,metrics=None,n_skip=0,**kwargs):
    inp,out = get_inp_out_size(db)
    model = SimpleQRNN(inp,out,**kwargs)
    
    cbs=[TimeSeriesRegularizer(alpha=alpha,beta=beta,modules=[model.rnn]),SaveModelCallback()]
    if early_stop > 0:
        cbs += [EarlyStoppingCallback(patience=early_stop)]
        
    skip = partial(SkipNLoss,n_skip=n_skip)
        
    if metrics is None: metrics=skip(fun_rmse)
        
    lrn = Learner(db,model,loss_func=skip(nn.MSELoss()),opt_func=ranger,metrics=metrics,cbs=cbs)
    return lrn

In [ ]:
QRNNLearner(db).fit(1)

epoch,train_loss,valid_loss,fun_rmse,time
0,12.823062,12.561305,3.534583,00:03


## TCN Learner
Performs better on multi input data. Higher beta values allow a way smoother prediction. Way faster then RNNs in prediction. 

In [ ]:
#export
@delegates(TCN, keep=True)
def TCNLearner(db,hl_depth=3,alpha=1,beta=1,early_stop=0,metrics=None,n_skip=None,**kwargs):
    n_skip = 2**hl_depth if n_skip is None else n_skip
    skip = partial(SkipNLoss,n_skip=n_skip)
    
    inp,out = get_inp_out_size(db)
    model = TCN(inp,out,hl_depth,**kwargs)
    
    cbs=[TimeSeriesRegularizer(alpha=alpha,beta=beta,modules=[model.conv_layers[-1]]),SaveModelCallback()]
    if early_stop > 0:
        cbs += [EarlyStoppingCallback(patience=early_stop)]
        
    if metrics is None: metrics=SkipNLoss(fun_rmse,n_skip)
        
    lrn = Learner(db,model,loss_func=skip(nn.MSELoss()),opt_func=ranger,metrics=metrics,cbs=cbs)
    return lrn

In [ ]:
TCNLearner(db).fit(1)

epoch,train_loss,valid_loss,fun_rmse,time
0,11.804644,12.262656,3.459962,00:03


## Autoregressive Learner

In [ ]:
#export
@delegates(TCN, keep=True)
def AR_TCNLearner(db,hl_depth=3,alpha=1,beta=1,early_stop=0,metrics=None,n_skip=None,**kwargs):
    n_skip = 2**hl_depth if n_skip is None else n_skip
    skip = partial(SkipNLoss,n_skip=n_skip)
    
    inp,out = get_inp_out_size(db)
    model = AR_Model(TCN(inp+out,out,hl_depth,**kwargs),ar=False,rf=n_skip)
    model.init_normalize(db.one_batch())
    
    cbs=[ARInitCB(),TimeSeriesRegularizer(alpha=alpha,beta=beta,modules=[model.model.conv_layers[-1]]),SaveModelCallback()]
    if early_stop > 0:
        cbs += [EarlyStoppingCallback(patience=early_stop)]
        
    if metrics is None: metrics=SkipNLoss(fun_rmse,n_skip)
        
    lrn = Learner(db,model,loss_func=nn.MSELoss(),opt_func=ranger,metrics=metrics,cbs=cbs)
    return lrn

NameError: name 'delegates' is not defined

In [ ]:
#export
@delegates(AR_GRU, keep=True)
def AR_GRULearner(db,alpha=0,beta=0,early_stop=0,metrics=None,n_skip=0,fname='model',**kwargs):
    skip = partial(SkipNLoss,n_skip=n_skip)
    
    inp,out = get_inp_out_size(db)
    model = AR_Model(AR_GRU(inp+out,out,**kwargs),ar=False,hs=True)
    model.init_normalize(db.one_batch())
    
    cbs=[ARInitCB(),TimeSeriesRegularizer(alpha=alpha,beta=beta,modules=[model.model.rnn]),SaveModelCallback()]
    if early_stop > 0:
        cbs += [EarlyStoppingCallback(patience=early_stop)]
        
    if metrics is None: metrics=SkipNLoss(fun_rmse,n_skip)
        
    lrn = Learner(db,model,loss_func=nn.MSELoss(),opt_func=ranger,metrics=metrics,cbs=cbs)
    return lrn

In [ ]:
#hide
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted 01_model.ipynb.
Converted 02_learner.ipynb.
Converted 10_performance_test.ipynb.
Converted 11_ProDiag.ipynb.
Converted 90_imu_example.ipynb.
Converted 93_mdt_benchmark.ipynb.
Converted 96_EEG.ipynb.
Converted 97_Battery.ipynb.
Converted index.ipynb.
